# How to Esay Going Text Classification

Dieses Juypter Notebook wurde für den Vortrag *Textklassifikation mit Machine Learning* im Rahmen [DGPuK-Methoden-Tagung 2018 in Ilmenau](https://www.tu-ilmenau.de/en/computational-communication-science/dgpuk-methodentagung-2018/) erstellt und enthält ergänzenden Quellcode in der Programmiersprache Python (Python 3).

### Inhalt

* [Daten einlesen](#first-bullet)
* [Modell auswählen](#second-bullet)
* [Daten vorbereiten und aufteilen](#third-bullet)
* [Modell trainieren](#4-bullet)
* [Modell testen](#5-bullet)
* [Modell evaluieren](#6-bullet)


### Daten einlesen <a name="first-bullet"></a>

Mit Hilfe der Bibliothek **Pandas** können Daten aus einem .csv-File einlesen werden. Alternativ kann natürlich auch eine andere Bibliothek oder ein anderes Dateiformat gewählt werden.

In [1]:
import pandas as pd #Pandas-Bibliothek aufrufen (zukünftig unter dem Kürzel pd)

In [2]:
file = open("beispielposts.csv") #.csv-File aus einem lokale Speicherort einlesen.

In [3]:
df = pd.read_table(file, sep=';', encoding="utf-8") #File als pandas-DataFrame im Objekt df speichern.

In [4]:
df.head(15) #Zeigt die ersten 15 Fälle (Zeilen) des Data Frames, plus Header.

,Text,Klasse
0,Der Grund ist der hohe Ausl?nderanteil. Die Ta...,1
1,Bis zu -24 Grad!!!! ?,1
2,Entsprechende Schritte w?rden in den kommenden...,1
3,"Die Hygiene-Center haben Toiletten, Duschen, W...",1
4,F?r die tollste Frau der Welt! <3,1
5,Mehr Platz f?r dich! :-*,1
6,"""Sie sagen, sch?rfere Waffengesetze verringern...",1
7,Ihr esst gerne Tiefk?hlpizza? Dann k?nnte es s...,1
8,Auf jeden Fall ges?ndere Besch?ftigung als Zoc...,1
9,Sonntagabend gemeinsam auf dem Sofa. ?? (via L...,1


In [7]:
len(df) #Anzahl der Fälle im Datensatz

10000

### Modell auswählen <a name="second-bullet"></a>

Als Beispiel benutzen wir einen *Naive Bayes Classifier*, der sich als solide Baseline für Textklassifikationsprobleme auf Basis von Bag-of-Words eignet. Aus der Python-Bibliothek **scikit-learn (sklearn)** für ML
importieren wir die Klasse `MultinomialNB` für multinominal verteilte Daten (Dokumentation [hier](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)).

In [8]:
from sklearn.naive_bayes import MultinomialNB #Lädt den Classifier MultinominalNB aus der sklearn-Bibliothek.


MultinomialNB() #Default Parameter Setting

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
model = MultinomialNB()

### Daten vorbereiten und aufteilen <a name="third-bullet"></a>

Oft müssen die Daten noch mehr oder weniger aufwendig bearbeitet werden, bevor sie dem Modell zugeführt werden können. Für Textdaten kann diese Aufgabe besonders langwierig sein. Textverarbeitung kann z.B. mit den Bibliotheken **NLTK** oder **SpaCy** umgesetzt werden . Aber auch die ML-Bibliothek scikit-learn kann einige (einfache) Vorverarbeitung leisten. 

Die (vorverarbeiteten) Textdaten müssen noch in eine Bag-of-Words-Repräsenation überführt werden. Diese entspricht im Prinzip der (gewichteten) Verteilung der einzelnen Wörter in den Dokumenten wieder. Diese Aufgabe übernimmen die _Vectorizer_. Für eine absolute Häufigkeitsverteilung kann in sklearn der `CountVectorizer` verwendet werden.

In [11]:
#Importiert den CountVectorizer aus der sklearn-Bibliothek.
from sklearn.feature_extraction.text import CountVectorizer

Die _Vectorizer_ in sklearn nehmen mehrere Argumente entgegen, die u.a. Informationen darüber enthalten, welche Wörter, Zeichen oder Wortgruppen berücksichtigt werden sollen.

In [21]:
#Die folgenden Argumente können diesem Vectorizer übergeben werden.
#In der Default-Einstellung werden beispielsweise keine Satzzeichen berückstichtigt.
#Und Großbuchstaben werden zu Kleinbuchstaben.
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [13]:
#Weißt dem Objekt vec den Vectorizer zu.
vec = CountVectorizer()

Machen wir einen kurzen Text anhand eines kleinen Textes. 

In [14]:
test = ["Vielen Dank für die Blumen, vielen Dank, wie lieb von dir."]

In [16]:
test_vector = vec.fit_transform(test)

In [17]:
#Der Feature-Vektor für den obigen Satz sieht nach der Vektorizierung wie folgt aus. 
# Nicht wundern, die Worter sind alphabetisch sortiert.

test_vector.toarray() 

array([[1, 2, 1, 1, 1, 1, 2, 1, 1]], dtype=int64)

In [19]:
#Mit der Pandas-Bibliothek auch in schön:
pd.DataFrame(test_vector.toarray(),columns=vec.get_feature_names())


,blumen,dank,die,dir,für,lieb,vielen,von,wie
0,1,2,1,1,1,1,2,1,1


Nun müssen die Daten noch aufgeteilt werden. Im ML gibt es neben der folgenden Methode auch Verfahren, die den Datensatz mehrfach in Trainings- und Testsets aufteilen und stabiliere Schätzungen ermöglichen (-> Cross Validation).

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["Text"], #Als erstes wird X übergeben, hier df["Text"].
                                                    df["Klasse"], #Als erstes Argument die Klasse y, hier df["Klasse"].
                                                    test_size=0.20, #20 % der Daten sollen Testdaten sein.
                                                    random_state=4321) #Hier Random Number für die Zufallszuweisung.

Die Vectorizer verfügen über die Methoden `fit` und `transform` (oder `fit_transform`, wenn man beides auf einmal ausführen möchte). 


In [32]:
X_train_vec = vec.fit_transform(X_train.astype("U"))

An den _Trainingsdaten_ wird der Vectorizer _gefittet_ und die Texte transformiert.





Im _Testdatensatz_ fittet der Vectorizer nicht(!) noch einmal an dem Vokabular, sondern _transformiert_ nur auf Basis der Features, die er aus dem Trainset kennt.


In [33]:
X_test_vec = vec.transform(X_test.astype("U"))

In [31]:
X_train_vec 
#Die Bag-of-Words Matrix enthält 8000 Zeilen (Anzahl der Dokumente) x 27140 Spalten (Anzahl der Wörter im Gesamtcorpus)

<8000x27140 sparse matrix of type '<class 'numpy.int64'>'
	with 174441 stored elements in Compressed Sparse Row format>

### Modell trainieren <a name="4-bullet"></a>

Mit der `fit`-Methode, über die alle Classifier in sklearn verfügen, "lernt" das Modell, jedoch nur am Trainset, das aus `X_train_vec` und `y_train` besteht. 

Möchte man sich die Rechenzeit ausgeben lassen, geht das im Jupyter Notenbook über `%time`.

In [34]:
model.fit(X_train_vec, y_train) # %time model.fit(X_train_vect, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Modell testen <a name="5-bullet"></a>

Das trainierte Modell sagt nun die Klassen für die Testdaten voraus `X_test_vec`. Die Ergebnisse werden im Objekt `y_pred` gespeichert.

In [35]:
y_pred = model.predict(X_test_vec)

In [36]:
y_pred[:10] #Die ersten 10 vorhersagen sehen wie folgt aus:

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Modell evaluieren <a name="6-bullet"></a>

Für die Modellevaluation werden die wahren Werte für Y aus dem Testset `y_test` mit den geschätzten Werten `y_pred` durch das Modell `model` abgeglichen. Alles was wir brauchen finden wir in sklearn bei *metrics*.

In [37]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [39]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted 0', 'Predicted 1'],
    index=['True 0', 'True 1']
)

,Predicted 0,Predicted 1
True 0,1562,32
True 1,366,40
